# Lab setup

This tutorial walks through the process of setting up a lab and the related chemical inventory, vessels, tools etc.

First, we log into the server. In this case we are logging into a local instance of ESCALATE

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import escalateclient
import importlib

In [2]:
importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
username = 'vshekar'
password = 'copperhead123'
client = escalateclient.ESCALATEClient(server_url, username, password)

In [4]:
len(client.get('material'))

{}
GET: OK. Found 195 results


195

Next, we set up organizations and they can be hierarchical. For example, we can create Nielson lab under University of Colorado

In [4]:
# Creating University of Colorado first
cu_info = { "description": "University of Colorado",
            "full_name": "University of Colorado at Boulder",
            "short_name": "CU",
            "address1": "Boulder, CO 80309",
            "address2": "",
            "city": "Boulder",
            "state_province": "CO",
            "zip": "80309",
            "country": "USA",
            "phone": '123456',
            "website_url": 'www.colorado.edu',
            }

cu_response = client.get_or_create(endpoint='organization', data=cu_info)
#cu_response = client.get(endpoint='organization', data=cu_info)
print(cu_response)

{'description': 'University of Colorado', 'full_name': 'University of Colorado at Boulder', 'short_name': 'CU', 'address1': 'Boulder, CO 80309', 'city': 'Boulder', 'state_province': 'CO', 'zip': '80309', 'country': 'USA', 'phone': '123456', 'website_url': 'www.colorado.edu'}
http://localhost:8000/api/organization/?description=University+of+Colorado&full_name=University+of+Colorado+at+Boulder&short_name=CU&address1=Boulder%2C+CO+80309&city=Boulder&state_province=CO&zip=80309&country=USA&phone=123456&website_url=www.colorado.edu
GET: OK. Found 1 results
[{'url': 'http://localhost:8000/api/organization/202cdb68-ed8b-4519-bf50-6e2c383a2ab6/', 'uuid': '202cdb68-ed8b-4519-bf50-6e2c383a2ab6', 'edocs': [], 'tags': [], 'notes': [], 'address1': 'Boulder, CO 80309', 'address2': '', 'city': 'Boulder', 'state_province': 'CO', 'zip': '80309', 'country': 'USA', 'phone': '123456', 'add_date': '2022-02-21T14:44:15.136884', 'mod_date': '2022-02-21T14:44:15.136914', 'description': 'University of Colorado

In [5]:
# Then creating the Nielson Lab
nielson_lab_data = { 
    'description': 'Nielson Lab', 
    'address1': 'Boulder, CO 80309', 
    'address2': '', 
    'city': 'Boulder', 
    'state_province': 'CO', 
    'zip': '80309', 
    'country': 'USA', 
    'phone': '123456',
    'full_name': 'Nielson Lab', 
    'short_name': 'NL', 
    'website_url': 'www.colorado.edu', 
    'parent': cu_response[0]['url']}

nl_response = client.get_or_create(endpoint='organization', data=nielson_lab_data)

# Creating a person entry to identify the owner
# Note: If a user account is created, a person entry is automatically created for that user

nielson_data = {
    "first_name": "James",
    "middle_name": "",
    "last_name": "Nielson",
    "address1": "",
    "address2": "",
    "city": "",
    "state_province": "",
    "zip": "",
    "country": "",
    "phone": "",
    "email": "",
    "title": "",
    "suffix": "",
    "organization": cu_response[0]['url']
}

nielson_response = client.get_or_create('person', data=nielson_data)
nielson_actor = client.get('actor', data={'person': nielson_response[0]['url']})

{'description': 'Nielson Lab', 'address1': 'Boulder, CO 80309', 'city': 'Boulder', 'state_province': 'CO', 'zip': '80309', 'country': 'USA', 'phone': '123456', 'full_name': 'Nielson Lab', 'short_name': 'NL', 'website_url': 'www.colorado.edu', 'parent': '202cdb68-ed8b-4519-bf50-6e2c383a2ab6'}
http://localhost:8000/api/organization/?description=Nielson+Lab&address1=Boulder%2C+CO+80309&city=Boulder&state_province=CO&zip=80309&country=USA&phone=123456&full_name=Nielson+Lab&short_name=NL&website_url=www.colorado.edu&parent=202cdb68-ed8b-4519-bf50-6e2c383a2ab6
GET: OK. Found 1 results
{'first_name': 'James', 'last_name': 'Nielson', 'organization': '202cdb68-ed8b-4519-bf50-6e2c383a2ab6'}
http://localhost:8000/api/person/?first_name=James&last_name=Nielson&organization=202cdb68-ed8b-4519-bf50-6e2c383a2ab6
GET: OK. Found 1 results
{'person': '81417ed7-e262-4888-a286-5ffb740a3758'}
http://localhost:8000/api/actor/?person=81417ed7-e262-4888-a286-5ffb740a3758
GET: OK. Found 1 results


Next, we set up an inventory for the lab

In [6]:
# Get value, get always returns a list, even if there is only one element to be returned
active_status = client.get_or_create(endpoint='status', data={'description': 'active'})

# Inventory details
nl_inventory = {
    "description": "Nielson Lab Inventory",
    "status": active_status[0]['url'], # Indicates active status
    "actor": nielson_actor[0]['url'],
    "owner": nielson_actor[0]['url'], # Indicates James Nielson as owner
    "operator": nielson_actor[0]['url'], # Indicates James Nielson as operator
    "lab": nielson_actor[0]['url'] # Associates inventory with Nielson Lab
}

nl_inventory_response = client.get_or_create(endpoint='inventory', data=nl_inventory)

{'description': 'active'}
http://localhost:8000/api/status/?description=active
GET: OK. Found 1 results
{'description': 'Nielson Lab Inventory', 'status': '4dda7679-043f-4876-bbe8-f59a650666b9', 'actor': 'a62bcf58-9736-499a-8937-19460323adcb', 'owner': 'a62bcf58-9736-499a-8937-19460323adcb', 'operator': 'a62bcf58-9736-499a-8937-19460323adcb', 'lab': 'a62bcf58-9736-499a-8937-19460323adcb'}
http://localhost:8000/api/inventory/?description=Nielson+Lab+Inventory&status=4dda7679-043f-4876-bbe8-f59a650666b9&actor=a62bcf58-9736-499a-8937-19460323adcb&owner=a62bcf58-9736-499a-8937-19460323adcb&operator=a62bcf58-9736-499a-8937-19460323adcb&lab=a62bcf58-9736-499a-8937-19460323adcb
GET: OK. Found 1 results


Then add chemicals to the inventory

In [8]:
import pandas as pd
df = pd.read_csv('MetalChalcogenideRxns.csv')
data = {
    'description': '',
    'consumable': True,
    'material_class': 'model',
}
for chemical in df['Product expected']:
    data['description'] = chemical
    client.post('material', data=data)


POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
